### Plot aligned variables for lrauv vehicles against combined data 
To run this Notebook follow instructions at https://github.com/mbari-org/auv-python.

Processed files must be available locally. Execute `uv run src/data/process_lrauv.py` with the `--no_cleanup` option to create them,e.g.:
```
src/data/process_lrauv.py -v --log_file pontus/missionlogs/2024/20240715_20240725/20240723T023501/202407230235_202407232319.nc4 --no_cleanup
```

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/data'));
if module_path not in sys.path:
    sys.path.append(module_path)
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import ipywidgets as widgets
import pandas as pd
from pathlib import Path
import netCDF4 as nc4
import logging
from nc42netcdfs import BASE_LRAUV_PATH

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Change to logging.DEBUG for more detailed output
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Enable bokeh extension for hvplot
hv.extension('bokeh')

# Get time coordinate for each variable by introspection
def get_time_coord(var):
    """Get the time coordinate name for a variable.
    
    Args:
        var: Either an xarray.DataArray or netCDF4.Variable
        
    Returns:
        str: Name of the time coordinate/dimension
    """
    # Check if it's an xarray DataArray (has .dims attribute)
    if hasattr(var, 'dims'):
        # xarray DataArray
        time_dims = [dim for dim in var.dims if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dims[0]
    elif hasattr(var, 'dimensions'):
        # netCDF4 Variable
        time_dims = [dim for dim in var.dimensions if 'time' in dim.lower()]
        return time_dims[0] if time_dims else var.dimensions[0]
    else:
        raise TypeError(f"Unsupported variable type: {type(var)}")

# Pick the auv_name
auv_name = widgets.Dropdown(
    options=[f for f in sorted(os.listdir(BASE_LRAUV_PATH)) if f != ".DS_Store"],
    description='auv_name:',
    disabled=False,
)
display(auv_name)

In [ ]:
# Pick the log file from the selected LRAUV directory
# Pattern: {lrauv_name}/missionlogs/{year}/{date_range}/{mission_start}/{log_file}.nc4

lrauv_name = auv_name.value
log_files = sorted(Path(BASE_LRAUV_PATH).glob(f"{lrauv_name}/missionlogs/*/*/*/*[0-9].nc4"))
log_file_options = [str(f.relative_to(BASE_LRAUV_PATH)) for f in log_files]

log_file_picker = widgets.Select(
    options=log_file_options,
    description='Log File:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(log_file_picker)

In [ ]:
# Load combined and aligned files for comparison
log_file = log_file_picker.value
log_path = Path(BASE_LRAUV_PATH) / log_file
log_stem = log_path.stem
log_dir = log_path.parent

# Read the combined and aligned netCDF files
combined_file = log_file.replace('.nc4', '_combined.nc4')
aligned_file = log_file.replace('.nc4', '_align.nc4')

combined_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, combined_file))
aligned_ds = xr.open_dataset(os.path.join(BASE_LRAUV_PATH, aligned_file))

logger.debug(f"Combined file: {combined_file}")
logger.debug(f"Aligned file: {aligned_file}")

# Show dataset details only in DEBUG mode
if logger.isEnabledFor(logging.DEBUG):
    logger.debug("\nCombined file:")
    display(combined_ds)
    logger.debug("\nAligned file:")
    display(aligned_ds)

def plot_combined_vs_aligned(variable_name):
    """Plot a variable from both combined and aligned files overlaid.
    
    Args:
        variable_name: Name of the variable from combined dataset
        
    Returns:
        Overlay plot of combined and aligned data
    """
    logger.debug(f"Processing variable: {variable_name}")
    
    # Get time coordinates
    combined_time_coord = get_time_coord(combined_ds[variable_name])
    aligned_time_coord = get_time_coord(aligned_ds[variable_name])
    logger.debug(f"Time coords - Combined: {combined_time_coord}, Aligned: {aligned_time_coord}")
    
    # Get units for ylabel
    try:
        units = combined_ds[variable_name].attrs.get('units', '')
    except (AttributeError, KeyError):
        units = ''
    
    # Create ylabel with format "name (units)"
    ylabel = f"{variable_name} ({units})" if units else variable_name
    
    # Create time series plots with bold colors and transparency for blending
    combined_plot = combined_ds[variable_name].hvplot.line(
        x=combined_time_coord, 
        label='Combined', 
        color='#28A745',  # Green
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    aligned_plot = aligned_ds[variable_name].hvplot.line(
        x=aligned_time_coord, 
        label='Aligned', 
        color='#DC3545',  # Red
        width=900, 
        height=400, 
        ylabel=ylabel, 
        title=log_file, 
        alpha=0.7
    )
    
    logger.debug("Plots created successfully")
    
    # Overlay the plots on the same axes (order determines legend order: Combined, Aligned)
    return combined_plot * aligned_plot

# Select multiple variables to plot from the combined dataset
variable_picker = widgets.SelectMultiple(
    options=[var for var in combined_ds.data_vars],
    description='Variables:',
    disabled=False,
    rows=15,
    layout=widgets.Layout(width='800px')
)
display(variable_picker)

In [ ]:
# Plot all selected variables with linked axes
plots = []
for variable_name in variable_picker.value:
    plot = plot_combined_vs_aligned(variable_name)
    plots.append(plot)
display(hv.Layout(plots).cols(1).opts(shared_axes=True))